In [2]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="5" color="red">ch11_데이터프레임과 시리즈(Pandas)_9절~13절</font></b>

# 9절. 데이터 그룹화 및 집계

In [4]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Groupby()
cf. pivo_talbe

In [26]:
# 1) 그룹화 방법 1
# df.groupby(by=[그룹핑할열])[출력할 열].기초통계함수()   # [출력할 열]을 생략하면 모든열 출력
# df.groupby(by=[그룹핑할열]).기초통계함수()[출력할 열]   # 가능하지만 잘 안씀
iris_grouped = iris.groupby(['Species'])
iris_grouped = iris.groupby(iris.Species)
iris_grouped = iris.groupby('Species')

display(iris_grouped.head(2))

for group, data in iris_grouped:
    print(group)
    display(data.sample(2).sort_index())

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
100,6.3,3.3,6.0,2.5,virginica
101,5.8,2.7,5.1,1.9,virginica


setosa


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5,5.4,3.9,1.7,0.4,setosa
28,5.2,3.4,1.4,0.2,setosa


versicolor


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
76,6.8,2.8,4.8,1.4,versicolor
97,6.2,2.9,4.3,1.3,versicolor


virginica


,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
120,6.9,3.2,5.7,2.3,virginica
142,5.8,2.7,5.1,1.9,virginica


In [6]:
iris.groupby(by='Species').mean()  

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [8]:
iris.groupby(by='Species').mean()[ ['Petal.Length','Petal.Width'] ]  # 없어보인다
iris.groupby(by='Species')[ ['Petal.Length','Petal.Width'] ].mean()  # 그나마 이렇게 쓴다. 통계함수를 맨 뒤로

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [28]:
iris_grouped_sum = iris.groupby('Species').median()
iris_grouped_sum.reset_index()    # 한번에 이 결과를 출력하고 싶다! 한큐에 해보자

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,5.0,3.4,1.50,0.2
1,versicolor,5.9,2.8,4.35,1.3
2,virginica,6.5,3.0,5.55,2.0


In [32]:
# as_index=True 가 기본값, 그룹화한 열이 자동으로 인덱스가 됨. False를 하면 새로운 인덱스가 생성
iris.groupby('Species', as_index=True) 
iris_grouped_sum2 = iris.groupby('Species', as_index=False) .sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [35]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [36]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') | 
                  (iris_grouped_sum2.Species== 'versicolor') ]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [ ]:
# 1) 그룹화 방법 2
# df.pivot_table

In [41]:
iris.groupby('Species')[['Petal.Length', 'Petal.Width']].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length', 'Petal.Width'],
                aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [43]:
iris.groupby('Species').count()  # 결측치를 제외한 값의 갯수
iris.pivot_table(index='Species',
#                values=['Petal.Length', 'Petal.Width'],
                aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


### 다중열로 그룹화

In [44]:
# iris로 다중열로 그룹화 하기 위해 데이터 컬럼 추가(num필드)
iris.shape

(150, 5)

In [50]:
import numpy as np
np.ravel([[x]*25 for x in range(6)]) 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [53]:
iris['num'] = np.ravel( [ [x] *25 for x in range(6) ] ) 
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [57]:
# 각 열의 임의의 데이터에 결측치를 할당
import random
print(random.sample(range(150),3))
for col in range(iris.shape[1]-2):
    iris.iloc[random.sample(range(150), np.random.randint(1,4)), col] = np.nan

[3, 107, 43]


In [58]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  147 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   149 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [62]:
iris.groupby(by=['Species', 'num'])[['Petal.Length', 'Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              25           24
versicolor 2              24           25
           3              25           25
virginica  4              25           25
           5              23           25

In [65]:
iris.pivot_table(index=['Species', 'num'],
                 values=['Petal.Length', 'Petal.Width'],
                 aggfunc='count'
                )

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              25           24
versicolor 2              24           25
           3              25           25
virginica  4              25           25
           5              23           25

## 9.2 레이블 인코딩

In [67]:
iris.drop('num', axis=1, inplace=True)

In [72]:
# 레이블 인코딩을 지원하는 클래스 불러오기
from sklearn.preprocessing import LabelEncoder  # LabelEncoder 왜 대문자? 클래스인 걸 짐작할수 있다
le = LabelEncoder()  # 객체 생성
iris['target'] = le.fit_transform(iris.Species) # Species 열을 학습하여 숫자로 값을 변경해줌
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와이드 포맷 <-> 롱포맷
- [판다스 공식문서의 melt](https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt)
- [Tidy Data(7page)](https://vita.had.co.nz/papers/tidy-data.pdf)

In [79]:
import pandas as pd
realstate = pd.read_csv('data/부동산_250213/전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv', encoding='cp949')
realstate.head(2)

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,2014년9월,2014년10월,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,19404,19759,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443
1,부산,8111,8111,9078,8965,9402,9501,9453,9457,9411,9258,9110,9208,9208,9204,9235,9279,9327,9345,9515,9559,9581


In [76]:
from statsmodels import datasets
airquality = datasets.get_rdataset('airquality',
                                  package='datasets').data
airquality.head()

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


## 10.1 melt()를 이용한 언피버팅

In [87]:
pd.melt(airquality, id_vars=['Month', 'Day']) # 아래와 같음
airquality.melt(id_vars=['Month', 'Day'])
# 만들어지는 데이터 프레임을 조금 더 상세하게 다루기
air_melted = airquality.melt(id_vars=['Month', 'Day'],
                           var_name='변수명',  # 기본값 : variable
                           value_name='값'    # 기본값 : value
                           )
air_melted = air_melted.sort_values(by = ['Month', 'Day'])

In [94]:
air_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [98]:
realstate_long = realstate.melt(id_vars=['지역'],
                              var_name='연월',
                              value_name='평당가(만원)')
realstate_long.iloc[:,1]

0      2013년12월
1      2013년12월
2      2013년12월
3      2013년12월
4      2013년12월
         ...   
352     2015년8월
353     2015년8월
354     2015년8월
355     2015년8월
356     2015년8월
Name: 연월, Length: 357, dtype: object

## 10.2 pivot_table()을 이용한 피벗팅(롱포맷->와이드포맷)

In [99]:
#air_melted를 와이드 포멧으로 변경
air_melted.head()

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0


In [105]:
airquality2 = air_melted.pivot_table(index=['Month', 'Day'],
                                   columns='변수명',
                                   values = '값'
                                  )
airquality2.reset_index(inplace=True)
airquality2.columns.name = None

ValueError: Names must be a list-like

In [109]:
airquality2.columns.name = None
airquality2

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3
...,...,...,...,...,...,...
148,9,26,30.0,193.0,70.0,6.9
149,9,27,NaN,145.0,77.0,13.2
150,9,28,14.0,191.0,75.0,14.3
151,9,29,18.0,131.0,76.0,8.0


# 11절. 데이터프레임(시리즈)에 함수 적용하기
- apply(func, axis) : 축 요소별(요소별) 함수 적용 (데이터프레임, 시리즈에서 사용 가능)
- applymap(func) : 각 요소별 함수 적용 (데이터프레임에서만 사용 가능)
- map(func, lis) : 각 요소별 함수 적용 (1차원 리스트, 시리즈에서만 가능)
## 11.1 apply()

In [114]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [116]:
def get_year(datestr):    
    return int(datestr.split('-')[0])
get_year = lambda datestr : int(datestr.split('-')[0])
get_month = lambda datestr : int(datestr.split('-')[1])

In [118]:
member['Year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member['Month'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,Year,Month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [127]:
X = iris.iloc[:,:-2]
# X의 열 평균
avg = X.mean()
avg

Sepal.Length    5.857823
Sepal.Width     3.061486
Petal.Length    3.720408
Petal.Width     1.204698
dtype: float64

In [130]:
X.iloc[0]-avg

Sepal.Length   -0.757823
Sepal.Width     0.438514
Petal.Length   -2.320408
Petal.Width    -1.004698
dtype: float64

In [132]:
# X의 모든 행에 대해 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x : round(x-avg, 2), axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.76,0.44,-2.32,-1.0
1,-0.96,-0.06,-2.32,-1.0
2,-1.16,0.14,-2.42,-1.0
3,-1.26,0.04,-2.22,-1.0
4,-0.86,0.54,-2.32,-1.0
...,...,...,...,...
145,0.84,-0.06,1.48,1.1
146,0.44,-0.56,1.28,0.7
147,0.64,-0.06,1.48,0.8
148,0.34,0.34,1.68,1.1


## 11.2 applymap()
- 모든열, 모든행에 함수 적용

In [145]:
X.applymap( np.round )

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.0,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,5.0,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map
- 1차원, 시리즈 사용가능

In [152]:
member["Month2"] = member["Birth"].map( lambda date : int(date.split('-')[1]) )
# 1차원만 되니까 axis는 필요 없지
member

,Name,Age,Email,Address,Birth,Year,Month,Month2
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1,1


In [154]:
# 연습하기(apply, map)
# df 의 연도월 데이터를 연도와 월을 분리하여 int 변환후 컬럼을 추가하라
df['연도'] = df['연월'].apply(lambda x : int(x.split('년')[0]))

In [156]:
df['월'] = df['연월'].apply(lambda x : int(x.split('년')[1].replace('월','')))
# df['월'] = df['연월'].apply(lambda x : int(x.split('년')[1][:-1]))  선생님 풀이. 더 간결하고 쉽다. 참고하자

In [157]:
df

,지역,연월,평당가(만원),연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12
...,...,...,...,...,...
352,전북,2015년8월,6580,2015,8
353,전남,2015년8월,6289,2015,8
354,경북,2015년8월,7037,2015,8
355,경남,2015년8월,7665,2015,8


In [159]:
new_df = df.drop(labels=['연도', '월'], axis=1)
new_df

,지역,연월,평당가(만원)
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098
...,...,...,...
352,전북,2015년8월,6580
353,전남,2015년8월,6289
354,경북,2015년8월,7037
355,경남,2015년8월,7665


In [ ]:
def split_date(date):
    year = int(date.split('년')[0])
    month = int(date.split('년')[1][:-1])
    new_df['연도'] = year
    new_df['월'] = month
    if new_df
    

In [ ]:
df['연도'] = df['연월'].apply(lambda x : int(x.split('년')[0]))

In [ ]:
df['월'] = df['연월'].apply(lambda x : int(x.split('년')[1].replace('월','')))

In [174]:
def yaer_month_append(df):
    'df를 받아 연도와 월 추가 후 return'
    df = df.copy()
    df['연도'] = int(df['연월'].split('년')[0])
    df['월'] = int(df['연월'].split('년')[1][:-1])
    return df


In [172]:
yaer_month_append(df.iloc[0])

지역               서울
연월         2013년12월
평당가(만원)       18189
연도             2013
월                12
Name: 0, dtype: object

In [176]:
df2 = df.apply(yaer_month_append, axis=1)

In [179]:
df2.drop('연월', axis=1)

,지역,평당가(만원),연도,월
0,서울,18189,2013,12
1,부산,8111,2013,12
2,대구,8080,2013,12
3,인천,10204,2013,12
4,광주,6098,2013,12
...,...,...,...,...
352,전북,6580,2015,8
353,전남,6289,2015,8
354,경북,7037,2015,8
355,경남,7665,2015,8


# 12절. 일괄변경하기(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치는 drop하거나, 평균값/중위값으로 대체, 그룹별 평균/중위값 대체 

In [181]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                  [3,       4, np.nan, 1],
                  [np.nan,  3, np.nan, 5],
                  [np.nan,  3, np.nan, 4]],
                 columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [183]:
# 모든 결측치 0으로 대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [184]:
# 결측치를 forward. 이전 값이 전달되어 대체. 이전 값 없으면 전달 안됨
df.fillna(method='ffill')  

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [185]:
# 결측치를 backfill. 다음 값이 전달되어 대체. 다음 값 없으면 전달 안됨
df.fillna(method='bfill')  

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [187]:
# 결측치를 특정 값으로 대체
values = {'A' : 85, 'B' : 80, 'C' : 75, 'D' : 90}
df.fillna(value= values)  # value= 는 첫째 매개변수라 안써도 되겠다

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [191]:
# 결측치를 열별 평균 값으로 대체
df.fillna(value= df.mean(axis=0))
# 결측치를 열별 중위 값으로 대체
df.fillna(value= df.median(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [198]:
# 결측치를 2번째 결측치까지만 대체
df.fillna(value= df.median(axis=0), limit=2)

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,NaN,3,NaN,4


## 12.2 replace(to_value, new_value, inplae=T/F)
- to_value를 new_value로 바꿔
- 결측치를 fillna로 쓴다면 이상치는 replace로 쓴다
- to_value에 정규표현식 이용